In [1]:
from scipy.io import loadmat, savemat
import numpy as np
import time
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
from keras import backend as K

from keras.layers import Input, Dense, Dropout
from keras.models import Model
from keras.models import Sequential
from keras.layers import Activation
from keras.wrappers.scikit_learn import KerasRegressor

from keras.optimizers import SGD, adam, nadam, Adagrad
from keras.regularizers import l1,l2

from keras.callbacks import EarlyStopping, CSVLogger


import os
import os.path
import sys
import argparse
import time
import csv

D:\Users\Vishwesh\Anaconda3\envs\deep_l\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16569389845189943434
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 5072204595
locality {
  bus_id: 1
}
incarnation: 12247086378580759003
physical_device_desc: "device: 0, name: GeForce GTX 970M, pci bus id: 0000:01:00.0"
]


In [3]:
def load_all():
    #input = loadmat('training_input_b6000.mat')
    #output = loadmat('training_output_10th_order.mat')
    
    #input = loadmat('training_input_b6000_no_outliers.mat')
    #output = loadmat('training_output_10th_order_no_outliers.mat')
    
    #input = loadmat('training_input_b9000_no_outliers.mat')
    #output = loadmat('training_output_b9000_10th_order_no_outliers.mat')
    
    input = loadmat('training_input_b6000_final_no_outliers.mat')
    output = loadmat('training_output_final_no_outliers.mat')
    
    #X = np.array(input['training_input_b9000_no_outliers'])
    #X = np.array(input['training_input_b6000_no_outliers'])
    #X = np.array(input['training_input_b6000'])
    #X = np.array(input['input_matrix_randomized'])
    X = np.array(input['training_input_b6000_final_no_outliers'])
    
    #y = np.array(output['output_matrix_randomized'])
    #y = np.array(output['training_output_10th_order'])
    #y = np.array(output['training_output_10th_order_no_outliers'])
    #y = np.array(output['training_output_b9000_10th_order_no_outliers'])
    y = np.array(output['training_output_final_no_outliers'])
    
    # Get dimensions of arrays
    x_size = X.shape
    print('Input Array Shape',x_size)
    y_size = y.shape
    print ('Output Array Shape',y_size)
    return X,y

In [4]:
# Defined Deep NN Function 

def build_nn2():
    model = Sequential()
    # Input layer with dimension 1 and hidden layer i with 128 neurons.
    model.add(Dense(45, input_shape=(45,)))
    model.add(Dense(400))
    model.add(Activation("relu"))
    # Hidden layer j with 64 neurons plus activation layer.
    model.add(Dense(66))
    model.add(Activation("relu"))
    # Hidden layer k with 64 neurons.
    model.add(Dense(200))
    # Output Layer.
    model.add(Dense(66))
 
    # Model is derived and compiled using mean square error as loss
    # function, accuracy as metric and gradient descent optimizer.
    model.compile(loss='mse', optimizer='RMSprop', metrics=['accuracy'])
    model.summary()
    return model
    
    
    
    
    #model = Sequential()
    #model.add(Dropout(0.3, input_shape=(X.shape[1],)))
    #model.add(Dense(180, activation='relu', W_regularizer=l1(),kernel_initializer='random_uniform'))
    #model.add(Dense(160, activation='relu', W_regularizer=l1()))
    #model.add(Dense(140, activation='relu', W_regularizer=l1()))
    #model.add(Dense(120, activation='relu', W_regularizer=l1()))
    #model.add(Dense(100, activation='relu', W_regularizer=l1()))
    #model.add(Dense(80, activation='relu', W_regularizer=l1()))
    #, init='normal' "Removed from the below line"
    #model.add(Dense(66))
    #opt = SGD(lr=1e-5, decay=1e-6)
    #opt = Adagrad(lr=0.01, epsilon=1e-08, decay=0.0)
    #mean_squared_error
    #categorical_crossentropy
    #model.compile(optimizer=opt, loss='mean_squared_error')
    #model.summary()
    #return model

In [5]:
# Originally number of epochs was set to 1000, currently at 10.
def train_nn(model, X, y, out_dir, val_size=0.2, n_epoch=1000):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    csv_logger = CSVLogger(os.path.join(out_dir, 'results.csv'))

    model.fit(X, y, epochs=n_epoch, batch_size=10000, verbose=1, shuffle=True, validation_split=val_size, callbacks=[csv_logger])
    return model
    
    #, batch_size=5,verbose=0,callbacks=[csv_logger]
    #regress = KerasRegressor(build_fn=model, epochs=n_epoch)
    
    #seed1 = 46
    #kfold = KFold(n_splits=3, random_state=seed1)
    #results = cross_val_score(regress, X, y, cv=kfold)
    #print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
    
    #regress.fit(X,y)
    #return regress


In [6]:
def save_estimate(model, X, y, out_file, indices):
    #y_pred = model.predict(X)

    #y_pred = y_scaler.inverse_transform(y_pred)
    #y = y_scaler.inverse_transform(y)

    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
     
    y_pred = model.predict(X)
    savemat(out_file, mdict={'out_pred': y_pred, 'out_true': y, 'indices': indices})

In [7]:
def save_test_set_prediction(model, out_file):

    #input_test = loadmat('testing_input_72_b6000.mat')
    #input_test = loadmat('testing_input_72_b9000.mat')
    #output_test = loadmat('testing_output_72_10th_order.mat')
    
    input_test = loadmat('test_sh_ln_dwmri.mat')
    output_test = loadmat('test_set_output_10th_order_final.mat')
    
    #X_f_t = np.array(input_test['testing_input_72_b9000'])
    X_f_t = np.array(input_test['test_sh_ln_dwmri.mat'])
    y_f_t = np.array(output_test['test_set_output_10th_order_final'])
    
    # Get dimensions of arrays
    x_size = X_f_t.shape
    print('Hist 72: Input Array Shape',x_size)
    y_size = y_f_t.shape
    print ('Output Hist 72: Array Shape',y_size)
    
    # Make Predictions
    pred = model.predict(X_f_t)
    
    # If output path does not exist, create it
    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    savemat(out_file, mdict={'out_pred': pred, 'out_true': y_f_t})
    

In [8]:
def save_vishabyte_predictions(model, out_file):
    input_test = loadmat('b2000_invivo_vish_avg_vol.mat')
    X_f_t = np.array(input_test['b2000_invivo_vish_avg_vol'])
    # Get dimensions of arrays
    x_size = X_f_t.shape
    print('Vishabyte: Input Array Shape',x_size)
    pred = model.predict(X_f_t)
    
    # If output path does not exist, create it
    out_path = os.path.dirname(out_file)
    if not os.path.exists(out_path):
        os.makedirs(out_path)
        
    # In vivo save matrix
    savemat(out_file, mdict={'out_pred': pred})

In [9]:
def pair_wise_reprod_check_2(model, out_file_1, out_file_2):
    # In vivo test
    ts4_3ta = loadmat('sh_ts01_3tb_feed.mat')
    ts4_3tb = loadmat('sh_ts01_austin_feed.mat')

    # Make numpy arrays
    X_3ta = np.array(ts4_3ta['sh_ts01_3tb_feed'])
    X_3tb = np.array(ts4_3tb['sh_ts01_austin_feed'])

    input_dummy_a = loadmat('in_vivo_dummy_a.mat')
    input_dummy_b = loadmat('in_vivo_dummy_b.mat')

    X_f_a = np.array(input_dummy_a['dummy_a'])
    X_f_b = np.array(input_dummy_b['dummy_b'])
    
    # Pred 3TA TS04
    pred = model.predict(X_3ta)
    savemat(out_file_1, mdict={'predicted':pred})
    #savemat('invivo_ts4_3ta.mat', mdict={'predicted':pred})
    
    # Pred 3TB TS04
    predi = model.predict(X_3tb)
    savemat(out_file_2, mdict={'predicted':predi})
    #savemat('invivo_ts4_3tb.mat', mdict={'predicted':predi})

In [10]:
def pair_wise_reprod_check_3(model, out_file_1, out_file_2):
    # In vivo test
    ts4_3ta = loadmat('orig_hcp_feed.mat')
    ts4_3tb = loadmat('retest_hcp_feed.mat')

    # Make numpy arrays
    X_3ta = np.array(ts4_3ta['orig_hcp_feed'])
    X_3tb = np.array(ts4_3tb['retest_hcp_feed'])

    #input_dummy_a = loadmat('in_vivo_dummy_a.mat')
    #input_dummy_b = loadmat('in_vivo_dummy_b.mat')

    #X_f_a = np.array(input_dummy_a['dummy_a'])
    #X_f_b = np.array(input_dummy_b['dummy_b'])
    
    # Pred 3TA TS04
    pred = model.predict(X_3ta)
    savemat(out_file_1, mdict={'predicted':pred})
    #savemat('invivo_ts4_3ta.mat', mdict={'predicted':pred})
    
    # Pred 3TB TS04
    predi = model.predict(X_3tb)
    savemat(out_file_2, mdict={'predicted':predi})
    #savemat('invivo_ts4_3tb.mat', mdict={'predicted':predi})

In [11]:
def pair_wise_reprod_check(model, out_file_1, out_file_2):
    # In vivo test
    ts4_3ta = loadmat('sh_ts04_3ta_feed.mat')
    ts4_3tb = loadmat('sh_ts04_3tb_feed.mat')

    # Make numpy arrays
    X_3ta = np.array(ts4_3ta['sh_ts04_3ta_feed'])
    X_3tb = np.array(ts4_3tb['sh_ts04_3tb_feed'])

    input_dummy_a = loadmat('in_vivo_dummy_a.mat')
    input_dummy_b = loadmat('in_vivo_dummy_b.mat')

    X_f_a = np.array(input_dummy_a['dummy_a'])
    X_f_b = np.array(input_dummy_b['dummy_b'])
    
    # Pred 3TA TS04
    pred = model.predict(X_3ta)
    savemat(out_file_1, mdict={'predicted':pred})
    #savemat('invivo_ts4_3ta.mat', mdict={'predicted':pred})
    
    # Pred 3TB TS04
    predi = model.predict(X_3tb)
    savemat(out_file_2, mdict={'predicted':predi})
    #savemat('invivo_ts4_3tb.mat', mdict={'predicted':predi})


In [14]:
def main():
    t0 = time.time()

    #args = parse_args()
    work_dir = os.getcwd()
    exp = 'Histo_b6000_HCP_test'
    itr = 150 # Estimated from CV


    print ("Loading data")
    X, y = load_all()
    indices = np.array(range(X.shape[0]))+1

    out_start_dir = os.path.join(work_dir,exp)
    #if not os.path.exists(out_start_dir):
    #    os.makedirs(out_start_dir)

    seed1 = 46
    seed2 = 23

    kf = KFold(n_splits=5, random_state=seed1,shuffle=True,)

    fold_num = 0
    model_D = build_nn2()
    
    for train, test in kf.split(X):
        # Set up training / testing data
        fold_num += 1
        X_train = X[train,:]
        y_train = y[train,:]
        X_test = X[test,:]
        y_test = y[test,:]
        indices_train = indices[train]
        indices_test = indices[test]
        #X_train, X_test, y_train, y_test, X_scaler, y_scaler = norm_data(X_train, X_test, y_train, y_test)



        # Want to submit this w/ 1000 different initializations
        np.random.seed(seed=seed2)

        # Deep NN
        print ("Training DNN with %d iterations, fold %d" % (itr, fold_num))
        out_dir_DNN = os.path.join(out_start_dir, str(fold_num))
        model_D = train_nn(model_D, X_train, y_train, out_dir_DNN, n_epoch=itr, val_size=0.2)

        print ("Saving training outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'training.csv')
        save_estimate(model_D, X_train, y_train, end_dir,indices_train)

        print ("Saving testing outputs")
        end_dir = os.path.join(out_start_dir, str(fold_num), 'testing.csv')
        save_estimate(model_D, X_test, y_test, end_dir,indices_test)

    #print ("Saving Vishabyte outputs")
    #end_dir = os.path.join(out_start_dir, str('Vishabyte_Test'), 'result_vol_invivo_vish_b2000.mat')
    #save_vishabyte_predictions(model_D, end_dir)

    print ("Saving Histology blind test outputs")
    end_dir = os.path.join(out_start_dir, str('Hist_Blind_72_Test'), 'result_b6000_testing_10th_order.mat')
    save_test_set_prediction(model_D, end_dir)

    #print ("Saving Pair Wise Reprod Results")
    #end_dir_1 = os.path.join(out_start_dir, str('ISMRM_TS01_Test'), 'result_scan_a.mat')
    #end_dir_2 = os.path.join(out_start_dir, str('ISMRM_TS01_Test'), 'result_scan_b.mat')
    #pair_wise_reprod_check_2(model_D, end_dir_1, end_dir_2)
    
    print ("Saving Pair Wise Reprod Results on HCP")
    end_dir_1 = os.path.join(out_start_dir, str('HCP_Test'), 'result_scan_a.mat')
    end_dir_2 = os.path.join(out_start_dir, str('HCP_Test'), 'result_scan_b.mat')
    pair_wise_reprod_check_3(model_D, end_dir_1, end_dir_2)
    
    print ("Saving network for this fold")
    net_dir_DNN = (os.path.join(out_start_dir, 'nets'))
    if not os.path.exists(net_dir_DNN):
        os.makedirs(net_dir_DNN)
    net_name = os.path.join(net_dir_DNN, exp+"_model_fold"+str(fold_num)+".h5")
    model_D.save(net_name)

    t1 = time.time()
    total_time = t1-t0
    print (total_time)

    with open(os.path.join(work_dir,exp,'time.csv'),"wb") as f:
        writer = csv.writer(f,delimiter=',')
        writer.writerow([total_time])

In [15]:
if __name__=='__main__':
    main()

Loading data
Input Array Shape (44541, 45)
Output Array Shape (44541, 66)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 45)                2070      
_________________________________________________________________
dense_7 (Dense)              (None, 400)               18400     
_________________________________________________________________
activation_3 (Activation)    (None, 400)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 66)                26466     
_________________________________________________________________
activation_4 (Activation)    (None, 66)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 200)               13400     
_________________________________________________________________
de

Epoch 50/150
28505/28505 [==============================] - 0s 13us/step - loss: 0.0058 - acc: 0.7700 - val_loss: 0.0105 - val_acc: 0.9481
Epoch 51/150
28505/28505 [==============================] - 0s 17us/step - loss: 0.0054 - acc: 0.7727 - val_loss: 0.0086 - val_acc: 0.9544
Epoch 52/150
28505/28505 [==============================] - 0s 15us/step - loss: 0.0053 - acc: 0.7722 - val_loss: 0.0100 - val_acc: 0.9517
Epoch 53/150
28505/28505 [==============================] - 0s 11us/step - loss: 0.0055 - acc: 0.7742 - val_loss: 0.0087 - val_acc: 0.9536
Epoch 54/150
28505/28505 [==============================] - 0s 12us/step - loss: 0.0056 - acc: 0.7741 - val_loss: 0.0097 - val_acc: 0.9552
Epoch 55/150
28505/28505 [==============================] - 0s 12us/step - loss: 0.0053 - acc: 0.7741 - val_loss: 0.0085 - val_acc: 0.9545
Epoch 56/150
28505/28505 [==============================] - 0s 12us/step - loss: 0.0055 - acc: 0.7686 - val_loss: 0.0104 - val_acc: 0.9478
Epoch 57/150
28505/28505 [=

Epoch 109/150
28505/28505 [==============================] - 0s 12us/step - loss: 0.0047 - acc: 0.7713 - val_loss: 0.0084 - val_acc: 0.9603
Epoch 110/150
28505/28505 [==============================] - 0s 12us/step - loss: 0.0045 - acc: 0.7818 - val_loss: 0.0078 - val_acc: 0.9559
Epoch 111/150
28505/28505 [==============================] - 0s 12us/step - loss: 0.0046 - acc: 0.7782 - val_loss: 0.0086 - val_acc: 0.9596
Epoch 112/150
28505/28505 [==============================] - 0s 12us/step - loss: 0.0047 - acc: 0.7810 - val_loss: 0.0079 - val_acc: 0.9603
Epoch 113/150
28505/28505 [==============================] - 0s 11us/step - loss: 0.0046 - acc: 0.7798 - val_loss: 0.0082 - val_acc: 0.9573
Epoch 114/150
28505/28505 [==============================] - 0s 13us/step - loss: 0.0045 - acc: 0.7818 - val_loss: 0.0079 - val_acc: 0.9597
Epoch 115/150
28505/28505 [==============================] - 0s 12us/step - loss: 0.0046 - acc: 0.7794 - val_loss: 0.0089 - val_acc: 0.9551
Epoch 116/150
28505/

Epoch 17/150
28506/28506 [==============================] - 0s 12us/step - loss: 0.0043 - acc: 0.7909 - val_loss: 0.0073 - val_acc: 0.9573
Epoch 18/150
28506/28506 [==============================] - 0s 16us/step - loss: 0.0044 - acc: 0.7832 - val_loss: 0.0095 - val_acc: 0.9360
Epoch 19/150
28506/28506 [==============================] - 0s 17us/step - loss: 0.0047 - acc: 0.7800 - val_loss: 0.0071 - val_acc: 0.9559
Epoch 20/150
28506/28506 [==============================] - 0s 13us/step - loss: 0.0042 - acc: 0.7886 - val_loss: 0.0079 - val_acc: 0.9586
Epoch 21/150
28506/28506 [==============================] - 0s 12us/step - loss: 0.0042 - acc: 0.7952 - val_loss: 0.0073 - val_acc: 0.9595
Epoch 22/150
28506/28506 [==============================] - 0s 12us/step - loss: 0.0043 - acc: 0.7895 - val_loss: 0.0079 - val_acc: 0.9554
Epoch 23/150
28506/28506 [==============================] - 0s 12us/step - loss: 0.0042 - acc: 0.7991 - val_loss: 0.0072 - val_acc: 0.9566
Epoch 24/150
28506/28506 [=

Epoch 76/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0040 - acc: 0.8015 - val_loss: 0.0071 - val_acc: 0.9564
Epoch 77/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0042 - acc: 0.7853 - val_loss: 0.0093 - val_acc: 0.8976
Epoch 78/150
28506/28506 [==============================] - 0s 15us/step - loss: 0.0044 - acc: 0.7730 - val_loss: 0.0068 - val_acc: 0.9548
Epoch 79/150
28506/28506 [==============================] - 0s 15us/step - loss: 0.0040 - acc: 0.7886 - val_loss: 0.0078 - val_acc: 0.9496
Epoch 80/150
28506/28506 [==============================] - 0s 15us/step - loss: 0.0040 - acc: 0.7946 - val_loss: 0.0071 - val_acc: 0.9548
Epoch 81/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0040 - acc: 0.7945 - val_loss: 0.0075 - val_acc: 0.9544
Epoch 82/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0041 - acc: 0.7956 - val_loss: 0.0074 - val_acc: 0.9538
Epoch 83/150
28506/28506 [=

...

Epoch 126/150
28506/28506 [==============================] - 0s 16us/step - loss: 0.0033 - acc: 0.8041 - val_loss: 0.0064 - val_acc: 0.9451
Epoch 127/150
28506/28506 [==============================] - 1s 18us/step - loss: 0.0035 - acc: 0.7724 - val_loss: 0.0078 - val_acc: 0.8824
Epoch 128/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0033 - acc: 0.7975 - val_loss: 0.0066 - val_acc: 0.9374
Epoch 129/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0032 - acc: 0.7949 - val_loss: 0.0073 - val_acc: 0.8925
Epoch 130/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0032 - acc: 0.8048 - val_loss: 0.0068 - val_acc: 0.9402
Epoch 131/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0033 - acc: 0.7939 - val_loss: 0.0074 - val_acc: 0.8935
Epoch 132/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0033 - acc: 0.8027 - val_loss: 0.0065 - val_acc: 0.9397
Epoch 133/150
28506/

Epoch 34/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0032 - acc: 0.7968 - val_loss: 0.0074 - val_acc: 0.8943
Epoch 35/150
28506/28506 [==============================] - 0s 15us/step - loss: 0.0032 - acc: 0.8000 - val_loss: 0.0065 - val_acc: 0.9457
Epoch 36/150
28506/28506 [==============================] - 0s 13us/step - loss: 0.0032 - acc: 0.7976 - val_loss: 0.0075 - val_acc: 0.8991
Epoch 37/150
28506/28506 [==============================] - 1s 19us/step - loss: 0.0033 - acc: 0.8027 - val_loss: 0.0065 - val_acc: 0.9335
Epoch 38/150
28506/28506 [==============================] - 0s 17us/step - loss: 0.0032 - acc: 0.8014 - val_loss: 0.0074 - val_acc: 0.9101
Epoch 39/150
28506/28506 [==============================] - 0s 13us/step - loss: 0.0032 - acc: 0.8031 - val_loss: 0.0066 - val_acc: 0.9415
Epoch 40/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0032 - acc: 0.7937 - val_loss: 0.0075 - val_acc: 0.8786
Epoch 41/150
28506/28506 [=

Epoch 93/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0031 - acc: 0.7926 - val_loss: 0.0077 - val_acc: 0.8936
Epoch 94/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0032 - acc: 0.8001 - val_loss: 0.0066 - val_acc: 0.9294
Epoch 95/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0031 - acc: 0.8047 - val_loss: 0.0072 - val_acc: 0.9221
Epoch 96/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0031 - acc: 0.8075 - val_loss: 0.0066 - val_acc: 0.9272
Epoch 97/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0032 - acc: 0.8029 - val_loss: 0.0076 - val_acc: 0.8879
Epoch 98/150
28506/28506 [==============================] - 0s 14us/step - loss: 0.0032 - acc: 0.7949 - val_loss: 0.0065 - val_acc: 0.9463
Epoch 99/150
28506/28506 [==============================] - 0s 15us/step - loss: 0.0031 - acc: 0.7965 - val_loss: 0.0073 - val_acc: 0.8981
Epoch 100/150
28506/28506 [

Hist 72: Input Array Shape (7272, 45)
Output Hist 72: Array Shape (7272, 66)
Saving Pair Wise Reprod Results on HCP
Saving network for this fold


ImportError: `save_model` requires h5py.